# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [81]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

city_data_df = city_data_df.dropna()

# Reset the index
city_data_df = city_data_df.reset_index(drop=True)

# Convert 'Humidity' to numeric, coercing errors
city_data_df['Humidity'] = pd.to_numeric(city_data_df['Humidity'], errors='coerce')

# Remove rows with NaN 'Humidity'
city_data_df = city_data_df[~city_data_df['Humidity'].isnull()]

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad bolivar,8.1222,-63.5497,30.95,58,79,3.74,VE,1689873220
1,1,adamstown,-25.0660,-130.1015,22.75,81,99,15.33,PN,1689873221
2,2,papatowai,-46.5619,169.4708,5.62,71,6,7.52,NZ,1689873221
3,3,port-aux-francais,-49.3500,70.2167,1.79,59,58,17.73,TF,1689873221
4,4,chonchi,-42.6128,-73.8092,13.40,83,82,4.62,CL,1689873221


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [90]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat',
                             geo=True, # Enable geographic coordinates
                             color='Humidity', # Color points by humidity
                             size='Humidity', # Size points by humidity
                             hover_cols=["City"], # Show city name on hover
                             title='City Weather Data'
                             )

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [83]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

ideal_cities = city_data_df[(city_data_df["Max Temp"] >= 21) & 
                            (city_data_df["Max Temp"] <= 27) & 
                            (city_data_df["Wind Speed"] <= 4.5) & 
                            (city_data_df["Cloudiness"] == 0)].dropna()
# Display ideal cities  
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,pinarbasi,38.7229,36.3931,21.64,28,0,2.84,TR,1689873224
129,129,laojunmiao,39.8333,97.7333,22.03,24,0,3.65,CN,1689873234
222,224,tambopata,-12.7333,-69.1833,26.94,50,0,1.54,PE,1689873249
244,247,colonia,50.9333,6.9500,24.42,41,0,2.06,DE,1689873253
285,288,saint-pierre,-21.3393,55.4781,21.82,78,0,2.57,RE,1689873044


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,pinarbasi,TR,38.7229,36.3931,28,
129,laojunmiao,CN,39.8333,97.7333,24,
222,tambopata,PE,-12.7333,-69.1833,50,
244,colonia,DE,50.9333,6.9500,41,
285,saint-pierre,RE,-21.3393,55.4781,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [91]:
# Set parameters to search for a hotel
radius = 10000
api_key = geoapify_key

# Print a message to indicate the hotel search is starting
print("Starting hotel search")

# Create an empty list to store hotel data
hotel_data = pd.DataFrame(columns=["City", "Place Name", "Place Latitude", "Place Longitude"])


# Iterate through the ideal cities DataFrame
for index, row in ideal_cities.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Construct the URL with the current city's latitude, longitude, and radius
    url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:{latitude},{latitude},{longitude},{longitude}&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783"

    print(f"URL: {url}")  # Print the URL

    # Make an API request using the constructed URL
    response = requests.get(url)

    # Convert the API response to JSON format
    response_data = response.json()

   #print(f"Response: {response_data}")  # Print the response

    # Check if any places were found
    if response_data.get("features"):
        # Iterate through the place features and extract relevant information
        for feature in response_data["features"]:
            properties = feature.get("properties")
            if properties and "name" in properties:
                place_name = properties["name"]
                place_lat = feature["geometry"]["coordinates"][1]
                place_lon = feature["geometry"]["coordinates"][0]
                
                new_row = pd.DataFrame({
                "City": [row['City']],
                "Place Name": [place_name],
                "Place Latitude": [place_lat],
                "Place Longitude": [place_lon]
                })

                hotel_data = pd.concat([hotel_data, new_row], ignore_index=True)
   
    else:
        print("Invalid data format for a place. Skipping...")

# Clean hotel data
hotel_data = hotel_data.dropna()

# Reset the index
hotel_data = hotel_data.reset_index(drop=True)

# Create a DataFrame from the hotel data

hotel_df = pd.DataFrame(hotel_data)


# Display the hotel DataFrame
print(hotel_df)


Starting hotel search
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:38.7229,38.7229,36.3931,36.3931&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:39.8333,39.8333,97.7333,97.7333&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
Invalid data format for a place. Skipping...
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:-12.7333,-12.7333,-69.1833,-69.1833&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
Invalid data format for a place. Skipping...
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:50.9333,50.9333,6.95,6.95&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
URL: https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect:-21.3393,-21.3393,55.4781,55.4781&limit=20&apiKey=d548c5ed24604be6a9dd0d989631f783
Invalid data format for a place. Skipping...
         City          

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [89]:
%%capture --no-display



# Configure the map plot
map_plot = hotel_df.hvplot.points('Place Longitude', 'Place Latitude',
                                  geo=True,  # Enable geographic coordinates
                                  hover_cols=["City", "Place Name", "Country"],  # Show city name, hotel name, and country on hover
                                  title='City Weather Data'
                                 )

# Display the map
map_plot 

:Points   [Place Longitude,Place Latitude]   (City,Place Name)